In [ ]:
# | default_exp typing

# Typings
> types used throughout this package

In [ ]:
# | export
from typing import NamedTuple
import tensorflow_probability.substrates.jax.distributions as tfd
from jaxtyping import Float, Array, Bool

In [ ]:
# | export
InitialState = Float[Array, "m"]
Observations = Float[Array, "n+1 p"]
States = Float[Array, "n+1 m"]

## [00_glssm.ipynb]

In [ ]:
# | export
class GLSSMState(NamedTuple):
    u: Float[Array, "n+1 m"]  # state bias
    A: Float[Array, "n m m"]  # state transition matrix
    D: Float[Array, "n m l"]  # noise embedding matrix
    Sigma0: Float[Array, "m m"]  # initial state covariance matrix
    Sigma: Float[Array, "n l l"]  # state covariance matrix


class GLSSMObservationModel(NamedTuple):
    v: Float[Array, "n+1 p"]  # observation bias
    B: Float[Array, "n+1 p m"]  # observation matrix
    Omega: Float[Array, "n+1 p p"]  # observation covariance matrix


class GLSSM(NamedTuple):
    u: Float[Array, "n+1 m"]  # state bias
    A: Float[Array, "n m m"]  # state transition matrix
    D: Float[Array, "n m l"]  # noise embedding matrix
    Sigma0: Float[Array, "m m"]  # initial state covariance matrix
    Sigma: Float[Array, "n l l"]  # state covariance matrix
    v: Float[Array, "n+1 p"]  # observation bias
    B: Float[Array, "n+1 p m"]  # observation matrix
    Omega: Float[Array, "n+1 p p"]  # observation covariance matrix

## [10_kalman_filter_smoother.ipynb]

In [ ]:
# | export


class FilterResult(NamedTuple):
    x_filt: Float[Array, "n+1 m"]
    Xi_filt: Float[Array, "n+1 m m"]
    x_pred: Float[Array, "n+1 m"]
    Xi_pred: Float[Array, "n+1 m m"]


class SmootherResult(NamedTuple):
    x_smooth: Float[Array, "n+1 m"]
    Xi_smooth: Float[Array, "n+1 m m"]

## [20_pgssm.ipynb]

In [ ]:
# | export


class PGSSM(NamedTuple):
    u: Float[Array, "n+1 m"]
    A: Float[Array, "n m m"]
    D: Float[Array, "n m l"]
    Sigma0: Float[Array, "m m"]
    Sigma: Float[Array, "n l l"]
    v: Float[Array, "n+1 p"]
    B: Float[Array, "n+1 p m"]
    dist: tfd.Distribution
    xi: Float[Array, "n+1 p"]


def to_states(model: GLSSM | PGSSM) -> GLSSMState:
    return GLSSMState(
        u=model.u, A=model.A, D=model.D, Sigma0=model.Sigma0, Sigma=model.Sigma
    )


def to_observation_model(model: GLSSM) -> GLSSMObservationModel:
    return GLSSMObservationModel(v=model.v, B=model.B, Omega=model.Omega)


## [40_importance_sampling.ipynb]

In [ ]:
# | export
class GLSSMProposal(NamedTuple):
    u: Float[Array, "n+1 m"]
    A: Float[Array, "n m m"]
    D: Float[Array, "n m l"]
    Sigma0: Float[Array, "m m"]
    Sigma: Float[Array, "n l l"]
    v: Float[Array, "n+1 p"]
    B: Float[Array, "n+1 p m"]
    Omega: Float[Array, "n+1 p p"]
    z: Float[Array, "n+1 p"]


def to_glssm(proposal: GLSSMProposal) -> GLSSM:
    return GLSSM(
        proposal.u,
        proposal.A,
        proposal.D,
        proposal.Sigma0,
        proposal.Sigma,
        proposal.v,
        proposal.B,
        proposal.Omega,
    )


class ConvergenceInformation(NamedTuple):
    converged: Bool
    n_iter: int
    delta: Float

## [45_cross_entropy_method.ipynb]

In [ ]:
# | export
class MarkovProposal(NamedTuple):
    mean: Float[Array, "n+1 m"]
    R: Float[Array, "n+1 m m"]
    J_tt: Float[Array, "n m m"]  # lower triangular
    J_tp1t: Float[Array, "n m m"]

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()